# Model
当今主流大模型从架构上大致可分为稠密（Dense）模型和混合专家模型（Mixture of Expert, MoE）模型。

稠密模型中所有参数在每次计算时都会参与运算；混合专家模型则将不同的“专家”模块组合，根据输入选择合适的专家处理，能在保证效果的同时减少计算量和参数量。

MiniMind 模型在Llama 3.1 的基础上设计，基于经典的Transformer Decoder-Only 架构。

## MiniMind Dense Model

In [8]:
import math
import struct
import inspect
import time
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from model.LMConfig import LMConfig
from typing import Any, Optional, Tuple, List
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import PreTrainedModel
from transformers.modeling_outputs import CausalLMOutputWithPast

### 均方根层归一化（RMSNorm）
RMSNorm 是对 LayerNorm 的改进，移除了均值项，可以视为 LayerNorm在均值为 0 时的特例。
* LayerNorm
$$
y = \frac{x- E(x)} {\sqrt {Var(x)+\epsilon}} * \gamma + \beta
$$
* RMS Norm
$$
a_i = \frac {a_i}{RMS(a)+\epsilon}*\gamma, \ where \ RMS(a) = \sqrt { \frac {1} {n} \sum_{i=1}^{n} a^2_i}
$$
RMS Norm在Layer Norm的基础上舍弃了中心化操作，仅用缩放进行归一化，其不改变数据原本的分布，有利于激活函数输出的稳定

In [9]:
class RMSNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-5):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def _norm(self, x):
        # x: [batch_size, seq_len, dim]
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)
    
    def forward(self, x):
        return self.weight * self._norm(x.float()).type_as(x)  # 输出与输入的数据类型一致，避免后续计算破坏混合精度训练

## Rotary Position Embedding(RoPE) 旋转位置编码
### Rotary Position Embedding, RoPE

旋转位置编码是一种能将相对位置信息集成到 self-attention 中, 进而提升 transformer 架构性能的位置编码方式, 和绝对位置编码相比, RoPE 具有很好的外推性, 是目前的主流位置编码方式.

外推性的解释, 通俗来说就是训练的时候限制了 512 的上下文长度，那么推理时如果面对超过该长度的文本，LLM 可能无法正确处理.

- **绝对位置编码**

绝对位置编码是早期 Transformer 架构采用的绝对位置编码方案，及那个每个位置映射为固定的向量表示.

$$f_{t:t\in\{q,k,v\}}(\boldsymbol{x}_i,i)=\boldsymbol{W}_{t:t\in\{q,k,v\}}(\boldsymbol{x}_i+\boldsymbol{p}_i)$$

其中编码向量 $p_i$ 的计算使用如下公式：

$$\boldsymbol{p}_{i,2t}=\sin\left(k/1000^{2t/d}\right), \boldsymbol{p}_{i,2t+1}=\cos\left(k/1000^{2t/d}\right)$$

正如其名，绝对位置编码只考虑了输入序列中的绝对位置关系，对于 token 之间的相对信息则没有纳入考虑.

- **旋转位置编码**

假定 query 和 key 的内积操作可以被函数 g 表示，该函数 g 的输入是词嵌入向量 $x_m, x_n$ 和它们之间的相对位置 $m-n$:

$$<f_q(x_m ,m), f_k(x_n, n)>=g(x_m, x_n, m, n)$$

旋转位置编码就是找到一个使上式成立的位置编码方式. 

出于认识的目的，我们省略复杂的数学推导，直接看 RoPE 的的结论：

存在这样一个正交矩阵：

$$\boldsymbol{R}_{\Theta,m}^d=\underbrace{\begin{pmatrix}\cos m\theta_0&-\sin m\theta_0&0&0&\cdots&0&0\\\sin m\theta_0&\cos m\theta_0&0&0&\cdots&0&0\\0&0&\cos m\theta_1&-\sin m\theta_1&\cdots&0&0\\0&0&\sin m\theta_1&\cos m\theta_1&\cdots&0&0\\\vdots&\vdots&\vdots&\vdots&\ddots&\vdots&\vdots\\0&0&0&0&\cdots&\cos m\theta_{d/2-1}&-\sin m\theta_{d/2-1}&-\sin m\theta_{d/2-1}\end{pmatrix}}_{\boldsymbol{W}_m}$$

其中，$\Theta=\left\{\theta_i=10000^{-2(i-1)/d},i\in[1,2,\ldots,d/2]\right\}$

我们可以将 query 和 key 的内积操作转换为与原始向量 $x$ 相关的以下等价形式：

$$
\boldsymbol{q}_m^\mathbf{T}\boldsymbol{k}_n=\left(\boldsymbol{R}_{\Theta,m}^d\boldsymbol{W}_q\boldsymbol{x}_m\right)^\mathbf{T}\left(\boldsymbol{R}_{\Theta,n}^d\boldsymbol{W}_k\boldsymbol{x}_n\right)=\boldsymbol{x}_m^\mathbf{T}\boldsymbol{W}_q\boldsymbol{R}_{\Theta,n-m}^d\boldsymbol{W}_k\boldsymbol{x}_n
$$

其中， $\boldsymbol{R}_{\Theta,n-m}^d=\left(\boldsymbol{R}_{\Theta,m}^d\right)^\mathbf{T}\boldsymbol{R}_{\Theta,n}^d$.

由于 $\boldsymbol{R}_{\Theta,m}^d$ 的稀疏性，直接使用矩阵乘法会浪费算力，因此代码中采用下述方式实现：

$$\boldsymbol{R}_{\Theta,m}^{d}\boldsymbol{x}=\begin{pmatrix}x_{0}\\x_{1}\\x_{2}\\x_{3}\\\vdots\\x_{d-2}\\x_{d-1}\end{pmatrix}\otimes\begin{pmatrix}\cos m\theta_{0}\\\cos m\theta_{0}\\\cos m\theta_{1}\\\cos m\theta_{1}\\\vdots\\\cos m\theta_{d/2-1}\\\cos m\theta_{d/2-1}\end{pmatrix}+\begin{pmatrix}-x_{1}\\x_{0}\\-x_{3}\\x_{2}\\\vdots\\-x_{d-1}\\x_{d-2}\end{pmatrix}\otimes\begin{pmatrix}\sin m\theta_{0}\\\sin m\theta_{0}\\\sin m\theta_{1}\\\sin m\theta_{1}\\\vdots\\\sin m\theta_{d/2-1}\\\sin m\theta_{d/2-1}\end{pmatrix}
$$

In [ ]:
def precompute_pos_cis